In [ ]:
import numpy as np
import re
from numpy.linalg import norm
from gensim.models import KeyedVectors
import ipdb

In [ ]:
emb_file = '/Users/jperez/research/nlp/word-embeddings/spanish-word-embeddings/examples/fasttext-sbwc.3.6.e20.vec'
we = KeyedVectors.load_word2vec_format(emb_file, limit=300000)

In [ ]:
def to_vector(texto):
    tokens = texto.split()
    vec = np.zeros(300)
    for word in tokens:
        # si la palabra está la acumulamos
        if word in we:
            vec += we[word]
    return vec / norm(vec)

In [ ]:
def similarity(texto_1, texto_2):
    vec_1 = to_vector(texto_1)
    vec_2 = to_vector(texto_2)
    sim = vec_1 @ vec_2
    return sim

In [ ]:
temas = [
    'pareja amor relación sexo sentimientos familia',
    'salud comida bebida ejercicio alcohol colesterol desayuno agua medico doctor desarreglos',
    'trabajo dinero oficina compañeros invertir'
]

textos = open('raw_data.txt').read().split('\n')[:-1]

In [ ]:
def asd(texto):
    texto = limpia(texto)
    amor = similarity(texto, temas[0])
    salud = similarity(texto, temas[1])
    dinero = similarity(texto, temas[2])
    return amor, salud, dinero

In [ ]:
def limpia(texto):
    texto = texto.lower()
    data = re.split('[, ;:/\.]+', texto)
    out = ' '.join(data)
    return out.strip()

In [ ]:
as_trsh = 0.01
st_trsh = 0.013
error_mult = 2.5

# heurística para dividr texto considerando el orden y la semántica
def divide(texto):
    # primero saca el último punto si es que el texto termina en punto
    texto = texto[:-1] if texto.endswith('.') else texto

    # divide en oraciones y devuelve los puntos
    data = texto.split('.')
    data = [x.strip() + '.' for x in data]
    
    #ipdb.set_trace()
    # decide cuando termina amor
    for salud_i in range(1,len(data)):
        a,s,_ = asd(data[salud_i])
        if s - a > as_trsh:
            break
            
    #ipdb.set_trace()
    for dinero_i in range(salud_i,len(data)):
        _,s,d  = asd(data[dinero_i])
        if d - s > st_trsh:
            break
            
    for error_i in range(dinero_i,len(data)):
        _,s,d  = asd(data[error_i])
        if s - d > error_mult*st_trsh:
            dinero_i = error_i + 1
        elif d - s > error_mult*st_trsh:
            break
            

    
    amor =  ' '.join(data[:salud_i])
    salud = ' '.join(data[salud_i:dinero_i])
    dinero = ' '.join(data[dinero_i:])

    return amor, salud, dinero    

In [ ]:
ejemplo = 'Busque las cualidades de su pareja y evite destacar sus defectos. Caminar es el mejor ejercicio. Practíquelo a diario. Cuide su trabajo, pues representa su sustento. No discuta tanto con sus compañeros de trabajo. Evite problemas.'

In [ ]:
divide(ejemplo)

In [ ]:
with open('../raw_amor.txt','w') as outamor, open('../raw_salud.txt','w') as outsalud, open('../raw_dinero.txt','w') as outdinero: 
    for texto in textos:
        if len(texto) < 10:
            continue
        amor, salud, dinero = divide(texto)
        if amor != '':
            outamor.write(amor + '\n')
        if salud != '':
            outsalud.write(salud + '\n')
        if dinero != '':
            outdinero.write(dinero + '\n')